# COVID-19 Overview
> Tracking coronavirus total cases, deaths and new cases

- comments: true
- author: Pratap Vardhan
- categories: [overview, interactive]
- image: images/covid-overview.png
- permalink: /covid-overview/

In [1]:
#hide
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [2]:
#hide

# FETCH
import getpass
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
base_url = '' if (getpass.getuser() == 'Pratap Vardhan') else base_url
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'css': base_url + 'overview.css',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read()
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_19-covid-{name}.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

In [3]:
#hide
        
# Confirmed, Recovered, Deaths
df = get_frame('Confirmed')
dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]

LAST_DATE_I = -1
# sometimes last column may be empty, then go backwards
for i in range(-1, -len(dt_cols), -1):
    if not df[dt_cols[i]].fillna(0).eq(0).all():
        LAST_DATE_I = i
        break

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('Deaths')
dft_recovered = get_frame('Recovered')
dfc_cases = dft_cases.groupby('Country/Region')[dt_today].sum()
dfc_deaths = dft_deaths.groupby('Country/Region')[dt_today].sum()
dfc_recovered = dft_recovered.groupby('Country/Region')[dt_today].sum()
dfp_cases = dft_cases.groupby('Country/Region')[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby('Country/Region')[dt_5ago].sum()
dfp_recovered = dft_recovered.groupby('Country/Region')[dt_5ago].sum()

In [4]:
#hide
df_table = (pd.DataFrame(dict(
    Cases=dfc_cases, Deaths=dfc_deaths, Recovered=dfc_recovered,
    PCases=dfp_cases, PDeaths=dfp_deaths, PRecovered=dfp_recovered))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
for c in 'Cases, Deaths, Recovered'.split(', '):
    df_table[f'{c} (+)'] = df_table[c] - df_table[f'P{c}']
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(15)

,Country/Region,Cases,Deaths,Recovered,PCases,PDeaths,PRecovered,Cases (+),Deaths (+),Recovered (+),Fatality Rate,Continent
0,China,81102,3241,69755,80945,3180,64196,157,61,5559,4.0,Asia
1,Italy,35713,2978,4025,17660,1266,1439,18053,1712,2586,8.3,Europe
2,Iran,17361,1135,5389,11364,514,2959,5997,621,2430,6.5,Asia
3,Spain,13910,623,1081,5232,133,193,8678,490,888,4.5,Europe
4,Germany,12327,28,105,3675,7,46,8652,21,59,0.2,Europe
5,France,9105,148,12,3678,79,12,5427,69,0,1.6,Europe
6,South Korea,8413,84,1540,7979,66,510,434,18,1030,1.0,Asia
7,US,7783,118,0,2179,47,12,5604,71,-12,1.5,North America
8,Switzerland,3028,28,15,1139,11,4,1889,17,11,0.9,Europe
9,United Kingdom,2642,72,67,802,8,19,1840,64,48,2.7,Europe


In [5]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Recovered', 'Cases (+)', 'Deaths (+)', 'Recovered (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu}
summary

{'updated': Timestamp('2020-03-18 00:00:00'),
 'since': Timestamp('2020-03-13 00:00:00'),
 'Cases': 214910,
 'Deaths': 8733,
 'Recovered': 83207,
 'Cases (+)': 69717,
 'Deaths (+)': 3329,
 'Recovered (+)': 12956,
 'China Cases': 81102,
 'China Deaths': 3241,
 'China Recovered': 69755,
 'China Cases (+)': 157,
 'China Deaths (+)': 61,
 'China Recovered (+)': 5559,
 'US Cases': 7783,
 'US Deaths': 118,
 'US Recovered': 0,
 'US Cases (+)': 5604,
 'US Deaths (+)': 71,
 'US Recovered (+)': -12,
 'EU Cases': 90439,
 'EU Deaths': 4011,
 'EU Recovered': 5450,
 'EU Cases (+)': 52135,
 'EU Deaths (+)': 2480,
 'EU Recovered (+)': 3707}

In [6]:
#hide
dft_ct_cases = dft_cases.groupby('Country/Region')[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,0,4,5,5,1,0
Albania,0,0,0,0,0,0,0,0,0,0,...,2,8,2,11,10,5,4,9,4,4
Algeria,0,0,0,0,0,0,0,0,0,0,...,1,0,0,4,2,11,11,6,6,14
Andorra,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,37,0
Antigua and Barbuda,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [7]:
#hide-input
template = Template(get_template(paths['overview']))
HTML('<div>' + 
    template.render(
    D=summary, table=df_table,
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd) +
    '</div>')

Interactive by [Pratap Vardhan](https://twitter.com/PratapVardhan)[^1]

[^1]: Source: ["2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). Link to [original notebook](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-overview.ipynb).